In [ ]:
import os

filename="outputs/ComfyUI_00309_.png"

file=os.path.basename(filename)
file

In [ ]:
from llama_cpp import Llama
import json

model_id="/Users/janghyeonbin/ai-companion/models/llm/gguf/Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"
model=Llama(model_id)

chat="안녕?"
response = model.create_chat_completion(
    messages=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": chat}
    ]
)

response["choices"][0]["message"]["content"]

In [ ]:
from diffusers import AutoPipelineForText2Image


In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5")

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
snapshot_download(repo_id="faridlazuarda/valadapt-llama-3.1-8B-it-korean", local_dir="./models/llm/loras/valadapt-llama-3.1-8B-it-korean")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path="/Volumes/EXDATA/models/LLM/transformers/llama/meta-llama__Llama-3.1-8B-Instruct"
lora_path="/Users/janghyeonbin/ai-companion/models/llm/loras/valadapt-llama-3.1-8B-it-korean"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
model.load_adapter(lora_path)